# Wildfires

Initialize Earthengine

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Download Data

In [1]:
from earthengine.init import *
from earthengine.fetch import *

ConnectionError: HTTPSConnectionPool(host='accounts.google.com', port=443): Max retries exceeded with url: /o/oauth2/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11ab50e20>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

Process and Analyze Data

In [2]:
FIRE_DF_PATH = "fire_df_08_05.csv"
WIND_DF_PATH = "wind_df_08_05.csv"

from timeinterp import *

time_interp(data_path=WIND_DF_PATH)

from preprocessing import *
preprocess(data_path=FIRE_DF_PATH)

from cross_correlation import *


Grid shape: 31x37
Lag: 0
Cutoff: (24, 4)
StopIteration on wind data
Lag: 10
Cutoff: (24, 4)


/Users/timothyliu/PycharmProjects/wildfires/cross_correlation.py:42: RuntimeWarning: invalid value encountered in divide
  norm_grid = np.nan_to_num(sum_grid / divide_grid)       # nan = divide by 0 (no correlation)


Lag: 20
Cutoff: (24, 4)
Lag: 30
Cutoff: (24, 4)
Lag: 40
Cutoff: (24, 4)
Lag: 50
Cutoff: (24, 4)
Lag: 60
Cutoff: (24, 4)
Lag: 70
Cutoff: (24, 4)
Lag: 80
Cutoff: (24, 4)
Lag: 90
Cutoff: (24, 4)
Lag: 100
Cutoff: (24, 4)
Lag: 110
Cutoff: (24, 4)
